This notebook is for testing of:
- Data transformation from a "cleaned, preprocessed, splited" pickle file 
- Model loading
- Model training

In [ ]:
PROJECT_DIR = "C:/Users/Uyen/Documents/nlp/Thesis/aspect-targeted-polarity-classification"

In [ ]:
import sys
import os

sys.path.append(PROJECT_DIR)
sys.path.append(os.path.join(PROJECT_DIR, "finetuning_and_classification"))


import utils
from utils_glue import (compute_metrics, convert_examples_to_features,
                        output_modes, processors)


In [ ]:
# Test data load from pkl
# Test function YNmilk_data_reform

sentences, aspect_term_sentiments =utils.YNmilk_data_reform("C:\\Users\\Uyen\\Documents\\nlp\\YNdata\\data_splited\\train.pickle")

sentences[0:5], aspect_term_sentiments[0:5]

In [ ]:
os.chdir(PROJECT_DIR)
!python prepare_YNmilk_datasets.py \
    --files "C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\train.pickle" \
    --output_dir data/transformed/yndata \
    --istrain

In [ ]:
!python prepare_YNmilk_datasets.py \
--files "C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\validate.pickle" \
--output_dir data/transformed/yndata 

In [ ]:
os.chdir(PROJECT_DIR)

MODEL_DIR = os.path.join(PROJECT_DIR, "data/models/phobert_base/huggingface_phobert_base")
DATA_DIR = os.path.join(PROJECT_DIR, "data/transformed/yndata")
OUTPUT_DIR = os.path.join(PROJECT_DIR, "YNData-ar-phobert-milk")

!python ./finetuning_and_classification/run_glue.py \
    --model_type="phobert" \
    --model_name_or_path=C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/models/phobert_base/huggingface_phobert_base \
    --do_train \
    # --evaluate_during_training \
    --logging_steps 100 --save_steps 200 --task_name="atsc" \
    --seed 42 \
    --do_lower_case \
    --data_dir="$DATA_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --max_seq_length=128 \
    --learning_rate 3e-5 \
    --per_gpu_eval_batch_size=32 \
    --gradient_accumulation_steps=1 \
    --max_steps=800 \
    --overwrite_output_dir --overwrite_cache --warmup_steps=120
    

In [ ]:
from transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer)


# config_class, model_class, tokenizer_class = (BertConfig, BertForSequenceClassification, BertTokenizer)
# config = config_class.from_pretrained(MODEL_DIR, num_labels=num_labels, finetuning_task=task_name)
# tokenizer = tokenizer_class.from_pretrained(MODEL_DIR, do_lower_case=True)
# model = model_class.from_pretrained(MODEL_DIR, from_tf=bool('.ckpt' in MODEL_DIR), config=config)


In [ ]:
!python prepare_YNmilk_datasets.py \
--files "C:\\Users\\Uyen\\Documents\\nlp\YNdata\\data_splited\\test.pickle" \
--output_dir data/transformed/yndata/testdata

In [ ]:
os.chdir(PROJECT_DIR)

!python ./finetuning_and_classification/run_glue.py \
    --model_type="phobert" \
    --do_eval \
    --model_name_or_path="C:/Users/Uyen/Documents/nlp/Thesis/aspect-targeted-polarity-classification/data/models/finetuned_phobert4classification/" \
    --task_name="atsc" \
    --output_dir="C:/Users/Uyen/Documents/nlp/Thesis/aspect-targeted-polarity-classification/data/models/finetuned_phobert4classification/"  \
    --do_lower_case \
    --data_dir=./data/transformed/yndata/testdata \
    --max_seq_length=128 \
    --per_gpu_eval_batch_size=32 \
    --overwrite_output_dir --overwrite_cache

In [ ]:
# text parser
import xml.etree.ElementTree as ET

filepath = "C:/Users/Uyen/Documents/nlp/thesis/aspect-targeted-polarity-classification/data/transformed/yndata/testdata/test.xml"

with open(filepath, 'rb') as f:
    sentence_elements = ET.parse(f).getroot().iter('sentence')

e_0 = list(sentence_elements)[0]
e_0.find('text').text, e_0.find('aspectTerms').find('aspectTerm').get('term')

In [24]:
# Test dataloader
# !pip install datasets

from datasets import load_dataset
import os

data_dir="C:/Users/Uyen/Documents/nlp/YNdata/data_test"
train_file = os.path.join(data_dir, "train.csv")
eval_file = os.path.join(data_dir, "dev.csv")

dataset = load_dataset('C:\\Users\\Uyen\\Documents\\nlp\\thesis\\aspect-targeted-polarity-classification\\finetuning_and_classification\\load_dataset_ar.py', data_files={"train": train_file, "validation": eval_file})

Using custom data configuration default-0365d4a76ec2245b
                                ['Quà tặng']
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False]
['Chức_năng tiêu_hóa', 'Nguồn_gốc xuất_xứ', 'Hấp_thụ', 'Dinh_dưỡng']
[False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
['Thành_phần']
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]
['Phát_triển trí_não']
[False, False, Fa

In [10]:
dataset["train"]["label"]

[31, 3, 32, 29, 25, 2, 30, 6]

In [ ]:
import os


# YN Data proposed
YN_DATA_HOME = "C:\\Users\\Uyen\\Documents\\nlp\\YNdata"

# Clean YN Data dataframe pickle files
YN_CLEAN = os.path.join(YN_DATA_HOME, "splitted")

# YN Data transfrom to xml to be run_glue adaptable, for training
YN_AR = os.path.join(YN_DATA_HOME, "ar_dataset")

YN_ATSC = os.path.join(YN_DATA_HOME, "atsc_dataset")

# YN Data transfrom to xml to be run_glue adaptable, for testing
YN_TEST = os.path.join(YN_DATA_HOME, "test")

# Aspect regconition model
ASPECT_RECOGNITION_MODEL_DIR = "./AR"

# Target-based sentiment classification model
TARGETED_POLARITY_CLASSIFICATION_MODEL_DIR = "/gdrive/MyDrive/Thesis/Models/TASC"

In [ ]:
TRAIN_FILE = os.path.join(YN_AR, 'train.csv')
EVAL_FILE = os.path.join(YN_AR, 'dev.csv')

In [ ]:
!python ./finetuning_and_classification/my_run.py      --model_name_or_path="vinai/phobert-base"     --do_train     --use_fast_tokenizer=False     --task_name="ar"      --max_seq_length="256"     --overwrite_cache=True     --pad_to_max_length=True     --train_file="C:\\Users\\Uyen\\Documents\\nlp\\YNdata\\ar_dataset\\train.csv"      --validation_file="C:\\Users\\Uyen\\Documents\\nlp\\YNdata\\ar_dataset\\dev.csv"      --output_dir="./AR_Model" --overwrite_output_dir 